#### Data Cleanup 2



In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import timedelta
from pandas_profiling import ProfileReport

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def generate_profile(df):
    '''generate pandas profile'''
 
    profile = ProfileReport(df)   
    profile.to_widgets()

In [4]:
combined = pd.read_csv('D:\cap\capstone2\data\interim\combined.csv', low_memory=False)

In [5]:
combined.head()

,Unnamed: 0,pilot_key,ev_id,ac_key,find_key,ev_date,far_part,second_pilot,crew_category,crew_age,crew_sex,med_certf,med_crtf_vldty,date_lst_med,category_no,cat_descript,subcategory_no,sub_descript,section_no,sec_descript,subsection_no,subsec_descript,modifier_no,mod_descript,Cause_Factor,ev_type,ev_country,mid_air,on_ground_collision,light_cond,ev_highest_injury,wx_cond_basic,flt_plan_filed,type_fly,phase_flt_spec,CERT_ATP,CERT_REC,CERT_CFI,CERT_COMM,CERT_FE,CERT_FORN,CERT_MIL,CERT_PRI,CERT_STU,CERT_SPRT,CERT_RMPT,CERT_NONE,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTR_AME,INSTR_ASE,INSTR_GLI,INSTR_GYRO,INSTR_HELI,INSTR_IAIR,INSTR_IHEL,INSTR_PLFT,INSTR_SPRT,INSTR_NONE,INSTRUM_APLN,INSTRUM_HELI,INSTRUM_PLFT,INSTRUM_NONE,OTHR_AIRS,OTHR_BALL,OTHR_GLI,OTHR_GYRO,OTHR_HELI,OTHR_PLFT,OTHR_SUAS,OTHR_NONE,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INST,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,GLDR-INST,GLDR-IRCV,GLDR-L24H,GLDR-L30D,GLDR-L90D,GLDR-PIC,GLDR-TOTL,INC-NO,LTA-INST,LTA-IRCV,LTA-L24H,LTA-L30D,LTA-L90D,LTA-PIC,LTA-TOTL,MAKE-INST,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INST,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INST,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,REL-NO,ROTO-INST,ROTO-IRCV,ROTO-L24H,ROTO-L30D,ROTO-L90D,ROTO-PIC,ROTO-TOTL,SENG-INST,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-INST,SIMU-IRCV,SIMU-L24H,SIMU-L30D,SIMU-L90D,SIMU-PIC,SIMU-TOTL
0,0,20001208X0880311,20001208X08803,20001208X088031,20001208X0880311,9/14/1997 0:00:00,91,N,PILOT,46.0,M,CL3,UNK,10/3/1995 0:00:00,2,Personnel issues,4,Action/decision,15,Info processing/decision,20,Decision making/judgment,44,Pilot,C,ACC,USA,N,N,DAYL,FATL,IMC,NONE,PERS,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,20001208X0880311,20001208X08803,20001208X088031,20001208X0880312,9/14/1997 0:00:00,91,N,PILOT,46.0,M,CL3,UNK,10/3/1995 0:00:00,3,Environmental issues,3,Conditions/weather/phenomena,50,Ceiling/visibility/precip,75,Below VFR minima,84,Decision related to condition,C,ACC,USA,N,N,DAYL,FATL,IMC,NONE,PERS,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,20001208X0880311,20001208X08803,20001208X088031,20001208X0880313,9/14/1997 0:00:00,91,N,PILOT,46.0,M,CL3,UNK,10/3/1995 0:00:00,3,Environmental issues,3,Conditions/weather/phenomena,50,Ceiling/visibility/precip,75,Below VFR minima,82,Effect on operation,C,ACC,USA,N,N,DAYL,FATL,IMC,NONE,PERS,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,20001208X0880311,20001208X08803,20001208X088031,20001208X0880314,9/14/1997 0:00:00,91,N,PILOT,46.0,M,CL3,UNK,10/3/1995 0:00:00,1,Aircraft,6,Aircraft oper/perf/capability,20,Performance/control parameters,12,Altitude,20,Not attained/maintained,C,ACC,USA,N,N,DAYL,F

In [6]:
combined['crew_age'].fillna(0, inplace=True)
combined['crew_age'] = combined['crew_age'].astype(int)

In [7]:
#subset pilots with age = 0  (no age reported)
zero_age = combined[combined['crew_age'] == 0]
zero_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3593 entries, 64 to 47608
Columns: 146 entries, Unnamed: 0 to SIMU-TOTL
dtypes: float64(111), int32(1), int64(6), object(28)
memory usage: 4.0+ MB


In [8]:
# fill in pilot age with average pilot age based on total flight hours

#define levels of pilot experience categories in bin of 250 hours
pilot0_max_hour = zero_age['ALL-TOTL'].max()
age_med_by_time = {} #holds median age for each time bracket
for hour_bin in range(1, int(pilot0_max_hour), 250):
    time_range = (hour_bin, hour_bin + 250)
    pop_age = combined[(combined['ALL-TOTL'] >= int(hour_bin)) & (combined['ALL-TOTL'] < (int(hour_bin) + 250))]
    age_med_by_time[time_range] = int(pop_age['crew_age'].median())
    
totl_idx = zero_age.columns.get_loc('ALL-TOTL')
age_idx = zero_age.columns.get_loc('crew_age')

for i in range(0, len(zero_age.index)):
    for key in age_med_by_time:
        if (zero_age.iloc[i,totl_idx] >= key[0]) & (zero_age.iloc[i,totl_idx] < key[1]):
            zero_age.iloc[i,age_idx] = age_med_by_time[key]
            

D:\Anaconda\envs\Capstone2\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [9]:
#aged_pilots contains pilots with imputed ages based on experience
aged_pilots = zero_age[zero_age['crew_age'] != 0]
aged_pilots.reset_index(inplace=True)
print(aged_pilots.info())
aged_pilots.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Columns: 147 entries, index to SIMU-TOTL
dtypes: float64(111), int32(1), int64(7), object(28)
memory usage: 112.3+ KB
None


,index,Unnamed: 0,pilot_key,ev_id,ac_key,find_key,ev_date,far_part,second_pilot,crew_category,crew_age,crew_sex,med_certf,med_crtf_vldty,date_lst_med,category_no,cat_descript,subcategory_no,sub_descript,section_no,sec_descript,subsection_no,subsec_descript,modifier_no,mod_descript,Cause_Factor,ev_type,ev_country,mid_air,on_ground_collision,light_cond,ev_highest_injury,wx_cond_basic,flt_plan_filed,type_fly,phase_flt_spec,CERT_ATP,CERT_REC,CERT_CFI,CERT_COMM,CERT_FE,CERT_FORN,CERT_MIL,CERT_PRI,CERT_STU,CERT_SPRT,CERT_RMPT,CERT_NONE,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTR_AME,INSTR_ASE,INSTR_GLI,INSTR_GYRO,INSTR_HELI,INSTR_IAIR,INSTR_IHEL,INSTR_PLFT,INSTR_SPRT,INSTR_NONE,INSTRUM_APLN,INSTRUM_HELI,INSTRUM_PLFT,INSTRUM_NONE,OTHR_AIRS,OTHR_BALL,OTHR_GLI,OTHR_GYRO,OTHR_HELI,OTHR_PLFT,OTHR_SUAS,OTHR_NONE,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INST,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,GLDR-INST,GLDR-IRCV,GLDR-L24H,GLDR-L30D,GLDR-L90D,GLDR-PIC,GLDR-TOTL,INC-NO,LTA-INST,LTA-IRCV,LTA-L24H,LTA-L30D,LTA-L90D,LTA-PIC,LTA-TOTL,MAKE-INST,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INST,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INST,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,REL-NO,ROTO-INST,ROTO-IRCV,ROTO-L24H,ROTO-L30D,ROTO-L90D,ROTO-PIC,ROTO-TOTL,SENG-INST,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-INST,SIMU-IRCV,SIMU-L24H,SIMU-L30D,SIMU-L90D,SIMU-PIC,SIMU-TOTL
0,1174,1174,20080729X0112411,20080729X01124,20080729X011241,20080729X0112411,7/28/2008 0:00:00,91,UNK,PILOT,60,M,CL2,VALID,5/15/2008 0:00:00,3,Environmental issues,2,Physical environment,20,Object/animal/substance,10,Pole,91,Contributed to outcome,,INC,USA,N,N,DAYL,NONE,VMC,NONE,BUS,Approach - VFR pattern - turn to base,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1175,1175,20080729X0112411,20080729X01124,20080729X011241,20080729X0112412,7/28/2008 0:00:00,91,UNK,PILOT,60,M,CL2,VALID,5/15/2008 0:00:00,2,Personnel issues,6,Task performance,10,Planning/preparation,10,Performance calculations,36,Flight crew,C,INC,USA,N,N,DAYL,NONE,VMC,NONE,BUS,Approach - VFR pattern - turn to base,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1176,1176,20080729X0112411,20080729X01124,20080729X011241,20080729X0112413,7/28/2008 0:00:00,91,UNK,PILOT,60,M,CL2,VALID,5/15/2008 0:00:00,3,Environmental issues,3,Conditions/weather/phenomena,10,Temp/humidity/pressure,20,High density altitude,81,Effect on equipment,,INC,USA,N,N,DAYL,NONE,VMC,NONE,BUS,Approach - VFR pattern - turn to base,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1177,1177,20080729X0112411,20080729X01124,20080729X011241,20080729X0112414,7/28/2008 0:00:00,91,UNK,PILOT,60,M,CL2,VALID,5/15/2008 0:00:00,1,Aircraft,

In [10]:
#backfill ages to combined df.

age_idx = combined.columns.get_loc('crew_age')
pilot_idx = combined.columns.get_loc('pilot_key')

for i in range (0, len(aged_pilots.index)):
    for r in range (0, len(combined.index)):
        if aged_pilots.iloc[i,2] == combined.iloc[r,pilot_idx]:
            combined.iloc[r,age_idx] = aged_pilots.iloc[i,10]
           

In [11]:
#Check how many pilots left with no age derived
zero_age = combined[combined['crew_age'] == 0]
zero_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3495 entries, 64 to 47608
Columns: 146 entries, Unnamed: 0 to SIMU-TOTL
dtypes: float64(111), int32(1), int64(6), object(28)
memory usage: 3.9+ MB


In [12]:
#mean age of pilot in overall dataset
age_avg_df = combined[combined['crew_age']!=0]
avg_age = int(age_avg_df['crew_age'].mean())
print(avg_age)

53


In [13]:
#backfill remaining 0 ages with df average.
for i in range (0, len(combined.index)):
    if combined.iloc[i,age_idx] == 0:
        combined.iloc[i,age_idx] = avg_age

In [14]:
#Check how many pilots left with no age imputed
zero_age = combined[combined['crew_age'] == 0]
zero_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Columns: 146 entries, Unnamed: 0 to SIMU-TOTL
dtypes: float64(111), int32(1), int64(6), object(28)
memory usage: 0.0+ bytes


In [15]:
#table is now adjusted with derived ages
combined['crew_age'].describe()

count    47643.000000
mean        53.432571
std         14.768896
min         14.000000
25%         44.000000
50%         54.000000
75%         64.000000
max        100.000000
Name: crew_age, dtype: float64

In [16]:
#cleanup milestone - save to csv 
combined.to_csv('D:\cap\capstone2\data\interim\df_aged.csv', index=False)

####   ***************************     Cleanup - Milestone  Save-Reload Data  *****************************

In [160]:
#Reload data
df = pd.read_csv('D:\cap\capstone2\data\interim\df_aged.csv', low_memory=False)

In [161]:
df['crew_age'].isna().sum(axis=0)

0

In [162]:
#generate_profile(eda)

In [163]:
df.rename(columns={'INSTR_AME':'INSTRUCT_AME', 'INSTR_ASE':'INSTRUCT_ASE', 'INSTR_GLI':'INSTRUCT_GLI', 
                    'INSTR_IAIR':'INSTRUCT_IAIR', 'INSTR_SPRT':'INSTRUCT_SPRT', 'INSTR_NONE':'INSTRUCT_NONE',
                    'ALL-INST': 'ALL-INSTRUM', 'GLDR-INST':'GLDR-INSTRUCT', 'MAKE-INST':'MAKE-INSTRUCT',
                    'MENG-INST':'MENG-INSTRUCT', 'NGHT-INST':'NGHT-INSTRUCT', 'SENG-INST':'SENG-INSTRUCT'}, inplace=True)

In [164]:
#after further consideration drop additional columns as they won't be needed in analysis
to_drop = ['ac_key', 'ev_id', 'find_key','second_pilot', 'ev_country', 'flt_plan_filed', 'phase_flt_spec', 'CERT_RMPT', 
           'OTHR_SUAS', 'SIMU-INST', 'SIMU-IRCV', 'SIMU-L24H', 'SIMU-L30D', 'SIMU-L90D', 'SIMU-PIC',
           'ROTO-INST', 'ROTO-IRCV', 'ROTO-L24H', 'ROTO-L30D', 'ROTO-L90D', 'ROTO-PIC', 'ROTO-TOTL',
           'OTHR_AIRS', 'OTHR_BALL', 'OTHR_GLI', 'OTHR_GYRO', 'OTHR_HELI', 'OTHR_PLFT', 'OTHR_SUAS', 'OTHR_NONE',
           'LTA-INST', 'LTA-IRCV', 'LTA-L24H', 'LTA-L30D', 'LTA-L90D', 'LTA-PIC', 'LTA-TOTL', 'Unnamed: 0', 
           'INSTR_PLFT', 'CERT_FE', 'INSTR_GYRO', 'INSTR_HELI', 'INSTR_IHEL','INSTRUM_HELI', 'INSTR_PLFT',
           'INC-NO','REL-NO', 'INSTRUM_PLFT']
df.drop(columns=to_drop, inplace=True)
df.head()

,pilot_key,ev_date,far_part,crew_category,crew_age,crew_sex,med_certf,med_crtf_vldty,date_lst_med,category_no,cat_descript,subcategory_no,sub_descript,section_no,sec_descript,subsection_no,subsec_descript,modifier_no,mod_descript,Cause_Factor,ev_type,mid_air,on_ground_collision,light_cond,ev_highest_injury,wx_cond_basic,type_fly,CERT_ATP,CERT_REC,CERT_CFI,CERT_COMM,CERT_FORN,CERT_MIL,CERT_PRI,CERT_STU,CERT_SPRT,CERT_NONE,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_GLI,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUCT_NONE,INSTRUM_APLN,INSTRUM_NONE,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,GLDR-INSTRUCT,GLDR-IRCV,GLDR-L24H,GLDR-L30D,GLDR-L90D,GLDR-PIC,GLDR-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL
0,20001208X0880311,9/14/1997 0:00:00,91,PILOT,46,M,CL3,UNK,10/3/1995 0:00:00,2,Personnel issues,4,Action/decision,15,Info processing/decision,20,Decision making/judgment,44,Pilot,C,ACC,N,N,DAYL,FATL,IMC,PERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0
1,20001208X0880311,9/14/1997 0:00:00,91,PILOT,46,M,CL3,UNK,10/3/1995 0:00:00,3,Environmental issues,3,Conditions/weather/phenomena,50,Ceiling/visibility/precip,75,Below VFR minima,84,Decision related to condition,C,ACC,N,N,DAYL,FATL,IMC,PERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0
2,20001208X0880311,9/14/1997 0:00:00,91,PILOT,46,M,CL3,UNK,10/3/1995 0:00:00,3,Environmental issues,3,Conditions/weather/phenomena,50,Ceiling/visibility/precip,75,Below VFR minima,82,Effect on operation,C,ACC,N,N,DAYL,FATL,IMC,PERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0
3,20001208X0880311,9/14/1997 0:00:00,91,PILOT,46,M,CL3,UNK,10/3/1995 0:00:00,1,Aircraft,6,Aircraft oper/perf/capability,20,Performance/control parameters,12,Altitude,20,Not attained/maintained,C,ACC,N,N,DAYL,FATL,IMC,PERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0
4,20001208X0880311,9/14/1997 0:00:00,91,PILOT,46,M,CL3,UNK,10/3/1995 0:00:00,1,Aircraft,6,Aircraft oper/perf/capability,20,Performance/control parameters,0,(general),20,Not attained/maintained,C,ACC,N,N,DAYL,FATL,IMC,PERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0


In [165]:
#analyze far_part data (type of operation)
df.far_part.value_counts()

91      35454
UNK        24
103         6
091K        4
Name: far_part, dtype: int64

In [166]:
#unable to impute UNK values -- drop records
df.drop(df[df['far_part'] == 'UNK'].index, inplace = True)
df.far_part.value_counts()

91      35454
103         6
091K        4
Name: far_part, dtype: int64

In [167]:
df['far_part'].isna().sum(axis=0)

12155

In [168]:
df['far_part'] = df['far_part'].fillna('delete')

In [169]:
df.drop(df[df['far_part'] == 'delete'].index, inplace = True)

In [170]:
df['far_part'].isna().sum(axis=0)

0

In [171]:
#Analyze crew category
df['crew_category'].isna().sum(axis=0)

293

In [172]:
df['crew_category'] = df['crew_category'].fillna('UNK')
df.crew_category.value_counts()

PILOT      33207
STUDENT     1942
UNK          315
Name: crew_category, dtype: int64

In [173]:
#using domain knowledge will impute any UNK with less than 60 hour of flt time as Student, otherwise Pilot
unk_low_time = df.loc[(df['ALL-TOTL'] <= 60) & (df['crew_category'] == 'UNK')]
unk_low_time.shape

(0, 100)

There are no Crew_category UNK records with less that 60 flight hours. Inpute all UNK records as pilots.

In [174]:
df['crew_category'] = df['crew_category'].replace(['UNK'], 'PILOT')
df.crew_category.value_counts()

PILOT      33522
STUDENT     1942
Name: crew_category, dtype: int64

In [175]:
df['crew_sex'].isna().sum(axis=0)

7640

In [176]:
#analyse crew_sex
df['crew_sex'] = df['crew_sex'].fillna('UNK')
df.crew_sex.value_counts()

M      26894
UNK     7640
F        930
Name: crew_sex, dtype: int64

In [177]:
df['med_certf'].isna().sum(axis=0)

375

In [178]:
df['med_certf'] = df['med_certf'].fillna('UNK')
df['med_certf'] = df['med_certf'].str.strip()
df.med_certf.value_counts()

CL3     19377
CL2      7931
CL1      3539
NONE     1918
SPRT     1752
UNK       542
BASC      405
Name: med_certf, dtype: int64

In [179]:
#sport pilots do now required a med cert, Basic is not an official medical certificate. Entries may have been done to reflect
#other information. Will condense SPRT/BASC to NONE.

df['med_certf'] = df['med_certf'].replace(['SPRT', 'BASC'],'NONE')
df.med_certf.value_counts()

CL3     19377
CL2      7931
NONE     4075
CL1      3539
UNK       542
Name: med_certf, dtype: int64

In [180]:
df['med_certf'].isna().sum(axis=0)

0

In [181]:
df['med_crtf_vldty'].isna().sum(axis=0)

293

In [182]:
df['med_crtf_vldty'] = df['med_crtf_vldty'].fillna('UNK')
df.med_crtf_vldty.value_counts()

VALID      29797
UNK         3636
INVALID     2031
Name: med_crtf_vldty, dtype: int64

In [183]:
df['med_crtf_vldty'].isna().sum(axis=0)

0

In [184]:
df['ev_date'].isna().sum(axis=0)

0

In [185]:
class3_invalid = df.loc[(df['med_certf'] == 'CL3') & (df['med_crtf_vldty'] == 'UNK')][['pilot_key', 
                                            'ev_date', 'date_lst_med', 'med_certf', 'med_crtf_vldty']]
class3_invalid.sample(20)

,pilot_key,ev_date,date_lst_med,med_certf,med_crtf_vldty
2442,20081015X3131711,10/6/2008 0:00:00,5/14/2008 0:00:00,CL3,UNK
5944,20090727X9052911,7/20/2009 0:00:00,NaN,CL3,UNK
2716,20081101X3391911,11/1/2008 0:00:00,12/30/2007 0:00:00,CL3,UNK
47285,20191112X1540811,11/11/2019 0:00:00,7/29/2019 0:00:00,CL3,UNK
8655,20100328X3101011,3/28/2010 0:00:00,4/30/2009 0:00:00,CL3,UNK
40651,20170906X3474511,9/2/2017 0:00:00,6/20/2017 0:00:00,CL3,UNK
8010,20100119X9520211,1/14/2010 0:00:00,6/1/2004 0:00:00,CL3,UNK
14722,20110805X2261011,8/5/2011 0:00:00,NaN,CL3,UNK
39690,20170706X9515511,7/2/2017 0:00:00,5/22/2017 0:00:00,CL3,UNK
21359,20130305X0530311,2/28/2013 0:00:00,7/24/2012 0:00:00,CL3,UNK


Will derive medical certificate validity for UNK values by creating and checking medical expirtion date (last medical +  medical validity time) vs. event date if medical expiration is before event date then UNK will be changed to INVALID.

In [186]:
df['date_lst_med'].isna().sum(axis=0)

3895

In [187]:
#replace date_last_medical NaN values with future date
df['date_lst_med'] = df['date_lst_med'].replace(np.nan, '01/01/9999')
df['date_lst_med'].isna().sum(axis=0)


0

In [188]:
df.sample(10)

,pilot_key,ev_date,far_part,crew_category,crew_age,crew_sex,med_certf,med_crtf_vldty,date_lst_med,category_no,cat_descript,subcategory_no,sub_descript,section_no,sec_descript,subsection_no,subsec_descript,modifier_no,mod_descript,Cause_Factor,ev_type,mid_air,on_ground_collision,light_cond,ev_highest_injury,wx_cond_basic,type_fly,CERT_ATP,CERT_REC,CERT_CFI,CERT_COMM,CERT_FORN,CERT_MIL,CERT_PRI,CERT_STU,CERT_SPRT,CERT_NONE,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_GLI,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUCT_NONE,INSTRUM_APLN,INSTRUM_NONE,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,GLDR-INSTRUCT,GLDR-IRCV,GLDR-L24H,GLDR-L30D,GLDR-L90D,GLDR-PIC,GLDR-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL
46918,20190916X7485111,9/16/2019 0:00:00,91,PILOT,47,M,CL2,VALID,3/14/2018 0:00:00,1,Aircraft,6,Aircraft oper/perf/capability,20,Performance/control parameters,23,Lateral/bank control,20,Not attained/maintained,C,ACC,N,N,DAYL,NONE,VMC,PERS,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1400.0,1500.0,0.0,0.0,2.0,15.0,70.0,9700.0,10350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,30.0,300.0,300.0,0.0,0.0,0.0,0.0,0.0,1550.0,1600.0,0.0,0.0,0.0,2.0,6.0,1900.0,2000.0,0.0,0.0,2.0,15.0,70.0,7300.0,7900.0,70.0
24064,20131019X2434811,10/18/2013 0:00:00,91,PILOT,54,UNK,CL2,VALID,6/1/2013 0:00:00,1,Aircraft,6,Aircraft oper/perf/capability,20,Performance/control parameters,12,Altitude,20,Not attained/maintained,C,ACC,N,N,NITE,FATL,VMC,BUS,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8496,20100317X8022111,3/15/2010 0:00:00,91,PILOT,68,M,CL3,VALID,5/18/2009 0:00:00,1,Aircraft,6,Aircraft oper/perf/capability,20,Performance/control parameters,20,Directional control,20,Not attained/maintained,C,ACC,N,N,DAYL,SERS,VMC,PERS,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.0,0.0,4.0,22.0,32.0,0.0,16450.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,22.0,32.0,0.0,143.0,0.0,0.0,0.0,0.0,0.0,0.0,16037.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,22.0,32.0,0.0,270.0,0.0
6540,20090831X0071111,8/30/2009 0:00:00,91,PILOT,58,UNK,CL3,UNK,3/11/2009 0:00:00,1,NaN,2,NaN,28,NaN,21,NaN,8,NaN,C,ACC,N,N,DAYL,NONE,VMC,PERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,293.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30176,20150518X3264011,4/12/2015 0:00:00,91,PILOT,52,UNK,NONE,INVALID,01/01/9999,3,Environmental issues,2,Physical environment,20,Object/animal/substance,22,Residence/building,91,Contributed to outcome,,ACC,N,N,DAYL,MINR,VMC,OWRK,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,495.0,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227.0,227.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12543,20110307X0490711,3/5/2011 0:00:00,91,PILOT,57,M,CL2,VALID,4/30/2010 0:00:00,2,Personnel issues,4,Action/decision,15,Info processing/decision,20,Decision making/judgment,44,Pilot,C,ACC,N,N,DAYL,NONE,VMC,PERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0

In [189]:
# create medical expiration date column and populate based on medical type


c1_valid_days_under40 = 365 
c1_valid_days_over40 =  int(365/2)
c2_valid_days = 365
c3_valid_days_over40  = 365 * 5
c3_valid_days_under40 = 365 * 2


med_cert_idx = df.columns.get_loc('med_certf')
age_idx = df.columns.get_loc('crew_age')
df.insert(9,'medical_exp_date', '')
df['medical_exp_date'] = pd.to_datetime(df['medical_exp_date'])
last_med_idx = df.columns.get_loc('date_lst_med')

for i in range (len(df)):
    if (df.iloc[i, med_cert_idx] == 'CL1') & (df.iloc[i, age_idx] >= 40):
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c1_valid_days_over40)
    elif (df.iloc[i, med_cert_idx] == 'CL1') & (df.iloc[i, age_idx] < 40):
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c1_valid_days_under40)
    elif (df.iloc[i, med_cert_idx] == 'CL2'): 
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c2_valid_days)
    elif (df.iloc[i, med_cert_idx] == 'CL3') & (df.iloc[i, age_idx] >= 40):
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c3_valid_days_over40)
    elif (df.iloc[i, med_cert_idx] == 'CL3') & (df.iloc[i, age_idx] < 40):
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c3_valid_days_under40)       
        

TypeError: can only concatenate str (not "datetime.timedelta") to str

Task: look at description of findings and filter out those not directly pertaining with pilots.

In [83]:
combined['cat_descript'].value_counts()

Personnel issues         20177
Aircraft                 16818
Environmental issues      9908
Organizational issues      544
Name: cat_descript, dtype: int64

In [84]:
org_records = combined.loc[combined['cat_descript'] == 'Organizational issues' , ['category_no', 'cat_descript', 'subcategory_no', 'sub_descript', 'section_no', 'sec_descript', 'subsection_no', 'subsec_descript', 'modifier_no', 'mod_descript', 'Cause_Factor', 'ev_type']]
org_records

,category_no,cat_descript,subcategory_no,sub_descript,section_no,sec_descript,subsection_no,subsec_descript,modifier_no,mod_descript,Cause_Factor,ev_type
9,4,Organizational issues,3,Support/oversight/monitoring,10,Training,30,Emergency proc training,66,Manufacturer,F,ACC
10,4,Organizational issues,2,Management,10,Policy/procedure,10,Availability of policy/proc,66,Manufacturer,F,ACC
19,4,Organizational issues,2,Management,10,Policy/procedure,15,Adequacy of policy/proc,99,Not specified,,ACC
26,4,Organizational issues,2,Management,10,Policy/procedure,15,Adequacy of policy/proc,99,Not specified,,ACC
41,4,Organizational issues,3,Support/oversight/monitoring,20,Oversight,10,Oversight of personnel,70,FAA/Regulator,,ACC
...,...,...,...,...,...,...,...,...,...,...,...,...
45557,4,Organizational issues,3,Support/oversight/monitoring,20,Oversight,15,Oversight of operation,70,FAA/Regulator,F,ACC
45558,4,Organizational issues,3,Support/oversight/monitoring,40,Enforcement,15,Operational procedures,70,FAA/Regulator,F,ACC
45559,4,Organizational issues,3,Support/oversight/monitoring,50,Safety programs,15,Adequacy of safety program,69,Operator,F,ACC
45560,4,Organizational issues,3,Support/oversight/monitoring,50,Safety programs,20,Adherence to safety program,69,Operator,F,ACC


In [85]:
org_records['mod_descript'].value_counts()

Operator                          170
Manufacturer                      136
FAA/Regulator                      99
ATC                                40
Other institution/organization     22
Not specified                      22
Maintenance provider               17
Airport                            14
Other government                    9
Meteorological service              7
Training organization               3
Equipment supplier                  3
Emergency services                  2
Name: mod_descript, dtype: int64

In [86]:
org_records['subsec_descript'].value_counts()

Adequacy of policy/proc           95
(general)                         75
Oversight of operation            49
Equipment design                  30
Oversight of personnel            23
Oversight of maintenance          22
Availability of policy/proc       20
Equipment manufacture             19
Equip certification/testing       18
Adequacy of safety program        14
Policy/procedure development      14
Initial training                  13
Maintenance records               13
Document/info production          11
Upgrade training                  10
Design of document/info           10
Standard operating practices       9
Between groups/organizations       9
Oversight of reg compliance        8
Safety                             8
Document/info verification         5
Availability of safety program     5
Emergency proc training            5
Equipment monitoring               4
Adherence to safety program        4
Regulatory requirements            4
Recurrent training                 3
P

For the most part,  organizational issues are not directly related specifically to pilot action and will be removed from the dataset. 

In [87]:
combined = combined[combined['cat_descript'] != 'Organizational issues']

In [88]:
env_records = combined.loc[combined['cat_descript'] == 'Environmental issues' , ['category_no', 'cat_descript', 'subcategory_no', 'sub_descript', 'section_no', 'sec_descript', 'subsection_no', 'subsec_descript', 'modifier_no', 'mod_descript', 'Cause_Factor', 'ev_type']]
env_records

,category_no,cat_descript,subcategory_no,sub_descript,section_no,sec_descript,subsection_no,subsec_descript,modifier_no,mod_descript,Cause_Factor,ev_type
1,3,Environmental issues,3,Conditions/weather/phenomena,50,Ceiling/visibility/precip,75,Below VFR minima,84,Decision related to condition,C,ACC
2,3,Environmental issues,3,Conditions/weather/phenomena,50,Ceiling/visibility/precip,75,Below VFR minima,82,Effect on operation,C,ACC
5,3,Environmental issues,3,Conditions/weather/phenomena,40,Wind,40,Crosswind,99,Not specified,F,ACC
13,3,Environmental issues,4,Task environment,20,Pressures/demands,30,Equipment/operational,83,Effect on personnel,F,ACC
14,3,Environmental issues,3,Conditions/weather/phenomena,60,Light condition,20,Dark,83,Effect on personnel,F,ACC
...,...,...,...,...,...,...,...,...,...,...,...,...
47626,3,Environmental issues,3,Conditions/weather/phenomena,10,Temp/humidity/pressure,40,Thermal lifting,81,Effect on equipment,C,ACC
47628,3,Environmental issues,2,Physical environment,20,Object/animal/substance,20,Tree(s),82,Effect on operation,C,ACC
47640,3,Environmental issues,2,Physical environment,30,Runway/land/takeoff/taxi surface,20,Soft surface,82,Effect on operation,C,ACC
47641,3,Environmental issues,2,Physical environment,30,Runway/land/takeoff/taxi surface,0,(general),82,Effect on operation,C,ACC


In [ ]:
env_records.sub_descript.unique()

In [ ]:
env_records.mod_descript.unique()

In [ ]:
#condense environmental mod_descripts

cat_idx = combined.columns.get_loc('cat_descript')
mod_idx = combined.columns.get_loc('mod_descript')

for i in range (0, len(combined)):
    if combined.iloc[i,cat_idx] == 'Environmental issues':
        if combined.iloc[i, mod_idx] == 'Ability to respond/compensate':
            combined.iloc[i, mod_idx] = 'Response/compensation'
        elif combined.iloc[i, mod_idx] == 'Timing of related info':
            combined.iloc[i, mod_idx] = 'Availability of related info'

All environmetal issues appear to be applicable to analysis and these records will be retained.

Analize caracteritics of findings with category aircraft.

In [ ]:
aircraft = combined.loc[combined['cat_descript'] == 'Aircraft' , ['category_no', 'cat_descript', 'subcategory_no', 'sub_descript', 'section_no', 'sec_descript', 'subsection_no', 'subsec_descript', 'modifier_no', 'mod_descript', 'Cause_Factor', 'ev_type']]

In [ ]:
aircraft.sub_descript.unique()

In [ ]:
aircraft.sec_descript.unique()

In [ ]:
aircraft.subsec_descript.unique()

In [ ]:
air_mod = aircraft.mod_descript.unique().tolist()
air_mod

In [ ]:
#Keep only those records related to pilot action

rec_keep = ['Not attained/maintained', 'Capability exceeded', 'Damaged/degraded', 'Incorrect use/operation', 'Attain/maintain not possible',
 'Unintentional use/operation', 'Fluid level', 'Inoperative', 'Fluid management', 'Not used/operated',  
 'Unneccessary use/operation', 'Simulated malf/failure', 'Unnecessary use/operation']

combined = combined[(combined['cat_descript'] == 'Aircraft') & (combined['mod_descript'].isin(rec_keep))]

In [ ]:
aircraft = combined[combined['cat_descript'] == 'Aircraft']

In [ ]:
aircraft.mod_descript.value_counts()

In [ ]:
#condense mod_descriptions into more general categories

cat_idx = combined.columns.get_loc('cat_descript')
mod_idx = combined.columns.get_loc('mod_descript')

for i in range (0, len(combined)):
    if combined.iloc[i,cat_idx] == 'Aircraft':
        if combined.iloc[i, mod_idx] == 'Unnecessary use/operation':
            combined.iloc[i, mod_idx] = 'Improper use/operation'
        elif combined.iloc[i, mod_idx] == 'Unneccessary use/operation':
            combined.iloc[i, mod_idx] = 'Improper use/operation'
        elif combined.iloc[i, mod_idx] == 'Incorrect use/operation':
            combined.iloc[i, mod_idx] = 'Improper use/operation'
        elif combined.iloc[i, mod_idx] == 'Fluid level':
            combined.iloc[i, mod_idx] = 'Fluid management'
        elif combined.iloc[i, mod_idx] == 'Damaged/degraded':
            combined.iloc[i, mod_idx] = 'Damaged/degraded/inoperative'
        elif combined.iloc[i, mod_idx] == 'Inoperative':
            combined.iloc[i, mod_idx] = 'Damaged/degraded/inoperative'
            
            

In [ ]:
aircraft = combined[combined['cat_descript'] == 'Aircraft']
aircraft.mod_descript.value_counts()

In [ ]:
personnel_records = combined.loc[combined['cat_descript'] == 'Personnel issues' , ['category_no', 'cat_descript', 'subcategory_no', 'sub_descript', 'section_no', 'sec_descript', 'subsection_no', 'subsec_descript', 'modifier_no', 'mod_descript', 'Cause_Factor', 'ev_type']]
personnel_records.sample(10)